In [35]:
def  upload_raw_data_to_s3(self):
    import os
    from src.data.s3_utils import S3Handler

    # Initialize S3 handler
    s3_handler = S3Handler()

    # List existing buckets
    buckets = s3_handler.list_buckets()
    print("Existing buckets:", buckets)

    # Define your project bucket name
    project_bucket = "olkoa-projects"

    # Create the bucket if it doesn't exist
    if project_bucket not in buckets:
        s3_handler.create_bucket(project_bucket)
        print(f"Bucket '{project_bucket}' created.")
    else:
        print(f"Bucket '{project_bucket}' already exists.")

    # Upload raw data directory to S3
    local_raw_data_dir = "data/Projects/Dans la vallée haha/valley/raw"
    s3_prefix = "Dans la vallée haha/valley/raw/"

    s3_handler.upload_directory(
        local_dir=local_raw_data_dir,
        bucket_name=project_bucket,
        s3_prefix=s3_prefix
    )
    print(f"Uploaded contents of '{local_raw_data_dir}' to 's3://{project_bucket}/{s3_prefix}'")

In [36]:
from src.data.s3_utils import S3Handler

In [37]:
# Initialize the handler
s3_handler = S3Handler()

In [38]:

# List buckets
buckets = s3_handler.list_buckets()
print(f"Available buckets: {buckets}")

Available buckets: ['highco', 'olkoa-celine', 'olkoa-joel', 'olkoa-projects']


In [39]:
project_bucket = "olkoa-projects"

if not project_bucket in buckets:
    # Create a new bucket
    s3_handler.create_bucket(project_bucket)


In [40]:
# Upload the JSON as a file
s3_handler.upload_directory(
    local_dir="data/Projects/Dans la vallée haha/valley/raw",
    bucket_name=project_bucket,
    s3_prefix="Dans la vallée haha/raw/"
)

INFO:src.data.s3_utils:File data/Projects/Dans la vallée haha/valley/raw/8830.eml uploaded to olkoa-projects/Dans la vallée haha/raw/8830.eml


Uploaded data/Projects/Dans la vallée haha/valley/raw/8830.eml to olkoa-projects/Dans la vallée haha/raw/8830.eml


In [41]:
# List buckets
buckets = s3_handler.list_buckets()
print(f"Available buckets: {buckets}")

Available buckets: ['highco', 'olkoa-celine', 'olkoa-joel', 'olkoa-projects']


In [43]:
s3_handler.list_directories(bucket_name=project_bucket)

AttributeError: 'S3Handler' object has no attribute 's3_client'

In [ ]:
# # Get the list of objects and extract just the keys
# objects = s3_handler.list_objects(bucket_name=project_bucket)
# object_keys = [obj['key'] for obj in objects]
# s3_handler.delete_objects(bucket_name=project_bucket, object_keys=object_keys)

## Check s3 joel mailbox

In [ ]:
s3_handler.list_objects(bucket_name="olkoa-joel", prefix="")

[{'key': '1743755186948115500_1/Archives/2018/1532012493613110001.eml',
  'size': 48138,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 19, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1643830307189110001.eml',
  'size': 1013158,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 18, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165495768110001.eml',
  'size': 121628,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 16, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165541631110001.eml',
  'size': 121518,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 19, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165569835110001.eml',
  'size': 121588,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 18, tzinfo=tzutc())},
 {'key': '1743755186948115500_1/Archives/2022/1654165593875110001.eml',
  'size': 121564,
  'last_modified': datetime.datetime(2025, 4, 11, 14, 16, 16

In [ ]:
len(_)

177914

In [ ]:
celine_json = s3_handler.list_objects(bucket_name="olkoa-celine", prefix="")
print(len(celine_json))
print(celine_json[:5])

19327
[{'key': 'Archive/1.eml', 'size': 1216884, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 24, tzinfo=tzutc())}, {'key': 'Archive/10.eml', 'size': 3812, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/2.eml', 'size': 23929, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/3.eml', 'size': 339717, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 25, tzinfo=tzutc())}, {'key': 'Archive/4.eml', 'size': 144422, 'last_modified': datetime.datetime(2025, 5, 21, 9, 46, 26, tzinfo=tzutc())}]
